In [1]:
import ftplib
ftp = ftplib.FTP('f31-preview.runhosting.com','4009006_DATOS','Rafa9999')

In [2]:
ftp.dir()

drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16 12:05 .
drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16 12:05 ..
-rw-r--r--   1 4009006_DATOS 1111        33182 Dec 16 12:05 books.xml
-rw-r--r--   1 4009006_DATOS 1111         3841 Dec 16 12:05 colors.json
-rw-r--r--   1 4009006_DATOS 1111       763509 May  3 21:07 OptaF24.xml


In [3]:
filename = 'OptaF24.xml'
ftp.retrbinary("RETR " + filename, open(filename,'wb').write)

'226 Transfer complete'

In [4]:
ftp.quit()

'221 Goodbye.'

In [5]:
import xml.etree.ElementTree as ET

tree = ET.parse("OptaF24.xml")

In [6]:
gameFile = tree.getroot()

In [11]:
gameFile[0].attrib

{'id': '360481',
 'away_team_id': '43',
 'away_team_name': 'Manchester City',
 'competition_id': '8',
 'competition_name': 'English Barclays Premier League',
 'game_date': '2011-08-21T16:00:00',
 'home_team_id': '30',
 'home_team_name': 'Bolton Wanderers',
 'matchday': '2',
 'period_1_start': '2011-08-21T16:00:38',
 'period_2_start': '2011-08-21T17:03:47',
 'season_id': '2011',
 'season_name': 'Season 2011/2012'}

In [15]:
lst = tree.findall('Game')
print('Número de partidos:',len(lst))

Número de partidos: 1


In [16]:
partidos = tree.findall("Game")

In [18]:
eventos = partidos[0].findall("Event")

In [46]:
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []
minute = []
second = []
half = []
team = []

for evento in eventos:
    if evento.attrib.get('type_id')== "1":
        half.append(evento.attrib.get('period_id'))
        minute.append(evento.attrib.get('min'))
        second.append(evento.attrib.get('sec'))
        x_origin.append(evento.attrib.get('x'))
        y_origin.append(evento.attrib.get('y'))
        outcome.append(evento.attrib.get('outcome'))
        team.append(evento.attrib.get('team_id'))
        
        qualifiers = evento.findall("Q")
        for q in qualifiers:
            if q.attrib.get("qualifier_id")=="140":
                x_destination.append(q.attrib.get("value"))
            elif q.attrib.get("qualifier_id")=="141":
                y_destination.append(q.attrib.get("value"))

In [52]:
import pandas as pd

pases = pd.DataFrame({'team':team,
                     'half':half,
                     'minute':minute,
                     'second':second,
                     'x_origin':x_origin,
                     'y_origin':y_origin,
                     'x_destination':x_destination,
                      'y_destination':y_destination,
                      'outcome':outcome
                     })
pases.team.replace({'43':'Manchester City','30':'Bolton Wanderers'},inplace=True)

pases

,team,half,minute,second,x_origin,y_origin,x_destination,y_destination,outcome
0,Manchester City,1,0,1,50.1,50.0,52.4,49.1,1
1,Manchester City,1,0,2,48.2,49.1,29.0,76.5,1
2,Manchester City,1,0,19,27.8,100.0,49.3,93.4,0
3,Bolton Wanderers,1,0,23,50.9,20.0,63.6,30.1,1
4,Manchester City,1,0,30,29.3,73.0,30.2,55.6,1
...,...,...,...,...,...,...,...,...,...
941,Bolton Wanderers,2,93,42,16.3,82.5,71.7,70.7,0
942,Manchester City,2,93,45,28.6,27.6,34.2,9.5,1
943,Manchester City,2,93,46,34.4,9.1,63.0,20.7,1
944,Manchester City,2,93,47,62.4,20.7,61.8,12.5,1


In [59]:
pases.outcome = pases.outcome.astype('int')

In [60]:
# ¿Qué equipo ha dado más pases buenos?
pases[['team','outcome']].groupby('team',as_index=False).sum()

,team,outcome
0,Bolton Wanderers,279
1,Manchester City,427


In [63]:
# Pases de más de 20 metros en el eje x
pases['x_distance'] = pases.x_destination.astype('float')-pases.x_origin.astype('float') 
pases.loc[(pases.x_distance>20) & (pases.outcome==1)].team.value_counts()

Bolton Wanderers    35
Manchester City     32
Name: team, dtype: int64